In [4]:
using Toms566
using BinDeps
using Optim

In [5]:
Pkg.update()

INFO: Updating METADATA...
INFO: Updating Toms566...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [132]:
#### Newton ####
#Solves 1,3,5,8,10,16, and 17 with <150 its

function obj(x)
    return (p.obj(x),p.grd(x),p.hes(x))
end

function newtmin1(obj, x0, b; maxIts = 1000, opTol = 1e-6)
    
    its = 0
    (f0,g0,h0) = obj(x0)
    x = x0
    (f,g,h) = obj(x)
    
    while norm(g,2) > b*norm(g0,2)*opTol && norm(g,2)>1e-8 && its < maxIts 
       
        E = eigfact(h)
        V = E[:vectors]
        Lam = diagm(max(E[:values], 1e-2))
        d = -V*inv(Lam)*transpose(V)*g
        
        alpha = 1
        mu  = 1e-2
        (fp,gp,hp) = obj(x+alpha*d)
        while fp > f + alpha*mu*dot(g,d)
            alpha = alpha/2
            (fp,gp,hp) = obj(x+alpha*d)
        end
        
        x = x + alpha*d
        (f,g,h) = obj(x)
        its = its + 1
        
    end
    
    if norm(g,2) < opTol && its < maxIts
        println("Way 2 go, homeskillet.")
    else
        println("Aw, shucks")
    end
    
    return (x, f, norm(g,2), its)
    
end

newtmin1 (generic function with 1 method)

In [135]:
p = Problem(3)
x0 = p.x0
(x, f, ng, its) = newtmin1(obj, x0, 0)

Way 2 go, homeskillet.


([0.3989561378221997,1.0000190842030112,6.831435020907098e-20],1.1279327696199563e-8,9.701953934166454e-11,2)

In [127]:
#### Newton with warmup ####
#solves 4,6,12

function obj(x)
    return (p.obj(x),p.grd(x),p.hes(x))
end

function newtmin2(obj, x0, wrmUp, b; maxIts = 1000, opTol = 1e-6)
    
    its = 0
    (f0,g0,h0) = obj(x0)
    x = x0
    (f,g,h) = obj(x)
    
   while wrmUp > 0 && norm(g,2) > opTol 
    
        (f,g,h) = obj(x)
        (sig, W) = eig(h)
        L = maximum(sig)
        x = x - (1/L)*g
        
        wrmUp = wrmUp - 1
        its = its + 1
        
    end
    
    while norm(g,2) > b*norm(g0,2)*opTol && norm(g,2)>1e-8 && its < maxIts 
       
        E = eigfact(h)
        V = E[:vectors]
        Lam = diagm(max(E[:values], 1e-2))
        d = -V*inv(Lam)*transpose(V)*g
        
        alpha = 1
        mu  = 1e-2
        (fp,gp,hp) = obj(x+alpha*d)
        while fp > f + alpha*mu*dot(g,d)
            alpha = alpha/2
            (fp,gp,hp) = obj(x+alpha*d)
        end
        
        x = x + alpha*d
        (f,g,h) = obj(x)
        its = its + 1
        
    end
    
    if norm(g,2) < opTol && its < maxIts
        println("Way 2 go, homeskillet.")
    else
        println("Aw, shucks")
    end
    
    return (x, f , norm(g,2), its)
    
end
    

newtmin2 (generic function with 1 method)

In [134]:
p = Problem(12)
x0 = p.x0
(x, f, normg, its) = newtmin2(obj, x0, 500, 0)

Way 2 go, homeskillet.


([19.90289986006735,-350.76114077033543,-389.3115556379986],32.83499999999998,4.031835563860223e-111,3)